# 第2回講義　宿題

## 課題：
* データセット（`text`）で単語の分散表現を学習して、与えられた単語ペア（`word_pairs`）の類似度スコアをcos類似度で計算してください。
* 計算した類似度スコアをsubmission.csvに出力して、submission.csvを提出してください。
* submission.csvのフォーマットはsample_submission.csvを参考にしてください。
* 学習用データセットはtext8というもので、事前に前処理済みの英語のデータセットです。ピリオドもなく文同士がすべて連結した単語の羅列になっているので注意してください。
* text8の全てを学習に使うと時間がかかりすぎるので、今回はその一部(100万単語)を訓練用データセットに使います。
* Word Similarity Taskで評価します。人手で評価した類似度スコアと予測した類似度スコアの相関をPearsonの相関係数によって計算して評価します。
* CBOWやSkipgramなど好きなアルゴリズムを使って構いません。意欲のある方はSubsamplingやHierarchical Softmax([元論文](https://arxiv.org/abs/1310.4546)参照)を実装してみたりsubwordを利用するなどその他の論文を参考にしてみてください。

## 注意：
* 辞書に含める単語の最低出現頻度は**3**としてください。

In [ ]:
# 必要であれば変更してください
import os
os.chdir('/root/userspace/chap2/')

In [ ]:
# データセット概要
with open("./data/text8", "r") as f:
    text = f.readline()
text = text.strip().split()
print("総単語数:", len(text))
print("総単語語彙数:", len(set(text)))
print("データ:", " ".join(text[:100]))
del text

In [ ]:
import numpy as np
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

PAD = 0 # <PAD>のID
UNK = 1 # <UNK>のID
PAD_TOKEN = '<PAD>' # paddingに使います
UNK_TOKEN = '<UNK>' # 辞書にない単語

word2id = {
    PAD_TOKEN: PAD,
    UNK_TOKEN: UNK,
}

class Vocab(object):
    def __init__(self, word2id={}):
        """
        word2id: 単語(str)をインデックス(int)に変換する辞書
        id2word: インデックス(int)を単語(str)に変換する辞書
        """
        self.word2id = dict(word2id)
        self.id2word = {v: k for k, v in self.word2id.items()}    

    def build_vocab(self, sentences, min_count=3):
        # 各単語の出現回数の辞書を作成する
        word_counter = {}
        for sentence in sentences:
            for word in sentence:
                word_counter[word] = word_counter.get(word, 0) + 1

        # min_count回以上出現する単語のみ語彙に加える
        for word, count in sorted(word_counter.items(), key=lambda x: -x[1]):
            if count < min_count:
                break
            _id = len(self.word2id)
            self.word2id.setdefault(word, _id)
            self.id2word[_id] = word

        # 語彙に含まれる単語の出現回数を保持する
        self.raw_vocab = {w: word_counter[w] for w in self.word2id.keys() if w in word_counter}

def load_data():
    with open("./data/text8") as f:
        line = f.readline()
        line = line.strip().split()
    return line

text = load_data()
text = text[:1000000]

vocab = Vocab(word2id=word2id)
vocab.build_vocab([text], min_count=3)

In [ ]:
word_pairs = []
with open("./data/sample_submission.csv", "r") as fin:
    for line in fin:
        line = line.strip().split(",")
        word1 = line[0]
        word2 = line[1]
        word_pairs.append([word1, word2])

In [ ]:
# データローダー
# WRITE ME

# モデル
# WRITE ME

# 学習
# WRITE ME

# 学習済みモデルの埋め込み行列
embedding_matrix = # WRITE ME ex) model.embedding.weight.data.cpu().numpy()

pred_scores = []
for pair in word_pairs:
    w1 = embedding_matrix[vocab.word2id[pair[0]]]
    w2 = embedding_matrix[vocab.word2id[pair[1]]]
    score = np.dot(w1, w2)/np.linalg.norm(w1, ord=2)/np.linalg.norm(w2, ord=2)
    pred_scores.append(score)

with open("./data/submission.csv", "w") as fout:
    for pair, score in zip(word_pairs, pred_scores):
        fout.write(pair[0] + "," + pair[1] + "," + str(score) + "\n")